In [1]:
%matplotlib inline

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from statsmodels.api import Logit
import patsy

from joblib import load, dump

from IPython.display import display

In [2]:
with pd.HDFStore('out/ModelData.20160529.h5') as cstore:
    df_first = cstore['first_author']
    df_last = cstore['last_author']
    
print df_first.shape, df_last.shape
df_first.columns

(41618369, 54) (41618369, 54)


Index([u'source_id', u'source_year', u'source_n_mesh', u'source_n_mesh_ex',
       u'source_is_eng', u'source_country', u'source_is_journal',
       u'source_is_review', u'source_is_case_rep', u'source_is_let_ed_com',
       u'source_T_novelty', u'source_V_novelty', u'source_PT_novelty',
       u'source_PV_novelty', u'source_ncites', u'source_n_authors', u'sink_id',
       u'sink_year', u'sink_n_mesh', u'sink_n_mesh_ex', u'sink_is_eng',
       u'sink_is_journal', u'sink_is_review', u'sink_is_case_rep',
       u'sink_is_let_ed_com', u'sink_T_novelty', u'sink_V_novelty',
       u'sink_PT_novelty', u'sink_PV_novelty', u'sink_n_authors', u'year_span',
       u'journal_same', u'mesh_sim', u'title_sim', u'lang_sim',
       u'affiliation_sim', u'pubtype_sim', u'cite_sim', u'author_sim',
       u'gender_sim', u'eth_sim', u'n_common_authors', u'gender', u'eth1',
       u'eth2', u'pos', u'pos_nice', u'sink_last_ncites', u'sink_prev_ncites',
       u'auth_last_npapers', u'auth_prev_papers', u'jj_

## First author

In [3]:
df_t_first = df_first.pivot_table(index="gender", columns="is_self_cite",
               values=["source_year", u'source_n_mesh', u'source_is_eng', 
                       u'source_is_journal', u'source_is_review',
                       u'source_is_case_rep', u'source_is_let_ed_com',
                       u'source_T_novelty', u'source_V_novelty', u'source_PT_novelty',
                       u'source_PV_novelty', u'source_ncites', u'source_n_authors',
                       u'sink_year', u'sink_n_mesh', u'sink_n_mesh_ex', u'sink_is_eng',
                       u'sink_is_journal', u'sink_is_review', u'sink_is_case_rep',
                       u'sink_is_let_ed_com', u'sink_T_novelty', u'sink_V_novelty',
                       u'sink_PT_novelty', u'sink_PV_novelty', u'sink_n_authors',
                       u'year_span', u'journal_same',
                       u'mesh_sim', u'title_sim', u'lang_sim',
                       u'affiliation_sim', u'pubtype_sim', u'cite_sim', u'author_sim',
                       u'gender_sim', u'eth_sim', u'n_common_authors',
                       u'sink_last_ncites', u'sink_prev_ncites',
                       u'auth_last_npapers', u'auth_prev_papers',
                       u'jj_sim', ],
                        aggfunc=[np.mean, np.std])
df_t_first

mean                                                      \
              source_year              source_n_mesh            source_is_eng   
is_self_cite            0            1             0          1             0   
gender                                                                          
-             2003.663025  2003.635203     13.492236  12.879459      0.984394   
F             2003.630762  2003.603407     13.340089  13.120937      0.986878   
M             2003.588473  2003.562598     12.566233  11.993291      0.978689   

                                                                               \
                       source_is_journal           source_is_review             
is_self_cite         1                 0         1                0         1   
gender                                                                          
-             0.994790          0.990094  0.985736         0.137451  0.140329   
F             0.991611          0.989822  0.984499         0.192086  0.172727   
M             0.986260          0.984037  0.974721         0.249040  0.260359   

                 ...                  std                              \
                 ...     sink_last_ncites            sink_prev_ncites   
is_self_cite     ...                    0          1                0   
gender           ...                                                    
-                ...           165.533983  14.423557      6089.993852   
F                ...           165.747492  13.854496      6061.960400   
M                ...           142.186787  15.854636      4931.702627   

                                                                      \
                        auth_last_npapers           auth_prev_papers   
is_self_cite          1                 0         1                0   
gender                                                                 
-             62.932480          3.841717  6.052002        29.212131   
F             53.677177          2.792324  4.535226        25.234161   
M             69.766346          4.848691  6.959813        60.236903   

                                                 
                             jj_sim              
is_self_cite           1          0           1  
gender                                           
-              73.219138  88.872976  232.483856  
F              45.843821  99.535612  246.924991  
M             100.963713  93.925320  190.487885  

[3 rows x 172 columns]

In [4]:
df_t_first.T.unstack(level=0).to_csv("First_Author_gender_self_cites_means.txt", sep="\t")

In [5]:
with pd.option_context('display.max_rows', 100,
                       'display.max_columns', 20,
                       'display.precision', 3
                      ):
    display(df_t_first.T.unstack(level=0))

gender                                    -                    F             \
                                       mean        std      mean        std   
                     is_self_cite                                             
source_year          0             2003.663      1.111  2003.631      1.112   
                     1             2003.635      1.114  2003.603      1.114   
source_n_mesh        0               13.492      5.859    13.340      5.463   
                     1               12.879      5.999    13.121      5.392   
source_is_eng        0                0.984      0.124     0.987      0.114   
                     1                0.995      0.072     0.992      0.091   
source_is_journal    0                0.990      0.099     0.990      0.100   
                     1                0.986      0.119     0.984      0.124   
source_is_review     0                0.137      0.344     0.192      0.394   
                     1                0.140      0.347     0.173      0.378   
source_is_case_rep   0                0.036      0.186     0.030      0.169   
                     1                0.011      0.107     0.012      0.107   
source_is_let_ed_com 0                0.010      0.100     0.010      0.101   
                     1                0.015      0.120     0.016      0.125   
source_T_novelty     0               19.846     12.504    20.965     12.666   
                     1               20.304     12.522    21.361     12.524   
source_V_novelty     0             3254.876  10964.933  3607.549  12431.441   
                     1             3298.214  11458.840  3632.433  24886.165   
source_PT_novelty    0                2.287      4.948     2.531      5.164   
                     1                2.333      4.960     2.540      5.089   
source_PV_novelty    0               12.492    678.392    24.137   1390.511   
                     1               19.853   1199.187    10.357    564.517   
source_ncites        0               45.780     66.675    53.320     94.552   
                     1               37.349     33.979    42.054     49.816   
source_n_authors     0                5.130      3.021     5.094      3.145   
                     1                5.176      4.281     5.182      3.538   
sink_year            0             1995.610      7.494  1995.628      7.401   
                     1             2000.468      3.569  2000.044      3.687   
sink_n_mesh          0               12.898      5.661    12.999      5.390   
                     1               13.219      6.051    13.518      5.356   
sink_n_mesh_ex       0               58.737     24.327    59.668     23.138   
                     1               60.834     26.808    62.792     23.686   
sink_is_eng          0                0.996      0.065     0.996      0.063   
                     1                0.995      0.068     0.995      0.073   
sink_is_journal      0                0.985      0.123     0.983      0.129   
                     1                0.989      0.105     0.988      0.108   
sink_is_review       0                0.146      0.353     0.148      0.355   
                     1                0.051      0.220     0.065      0.246   
sink_is_case_rep     0                0.030      0.171     0.028      0.166   
                     1                0.015      0.120     0.015      0.121   
sink_is_let_ed_com   0                0.016      0.124     0.017      0.130   
                     1                0.011      0.106     0.012      0.107   
sink_T_novelty       0               16.561     11.402    17.127     11.490   
                     1               18.326     11.820    19.033     11.811   
sink_V_novelty       0             2445.306   9045.170  2521.263   7979.960   
                     1             2514.894   6996.478  2669.965  12369.468   
sink_PT_novelty      0                1.786      4.393     1.853      4.443   
                     1                1.722      4

## Last author

In [6]:
df_t_last = df_last.pivot_table(index="gender", columns="is_self_cite",
               values=["source_year", u'source_n_mesh', u'source_is_eng', 
                       u'source_is_journal', u'source_is_review',
                       u'source_is_case_rep', u'source_is_let_ed_com',
                       u'source_T_novelty', u'source_V_novelty', u'source_PT_novelty',
                       u'source_PV_novelty', u'source_ncites', u'source_n_authors',
                       u'sink_year', u'sink_n_mesh', u'sink_n_mesh_ex', u'sink_is_eng',
                       u'sink_is_journal', u'sink_is_review', u'sink_is_case_rep',
                       u'sink_is_let_ed_com', u'sink_T_novelty', u'sink_V_novelty',
                       u'sink_PT_novelty', u'sink_PV_novelty', u'sink_n_authors',
                       u'year_span', u'journal_same',
                       u'mesh_sim', u'title_sim', u'lang_sim',
                       u'affiliation_sim', u'pubtype_sim', u'cite_sim', u'author_sim',
                       u'gender_sim', u'eth_sim', u'n_common_authors',
                       u'sink_last_ncites', u'sink_prev_ncites',
                       u'auth_last_npapers', u'auth_prev_papers',
                       u'jj_sim', ],
                        aggfunc=[np.mean, np.std])
df_t_last

mean                                                      \
              source_year              source_n_mesh            source_is_eng   
is_self_cite            0            1             0          1             0   
gender                                                                          
-             2003.668537  2003.669454     12.921096  13.370146      0.978522   
F             2003.634251  2003.616251     12.995415  13.705126      0.984778   
M             2003.597514  2003.589243     12.822461  13.196806      0.980800   

                                                                               \
                       source_is_journal           source_is_review             
is_self_cite         1                 0         1                0         1   
gender                                                                          
-             0.995681          0.986633  0.989529         0.171401  0.120635   
F             0.993584          0.986533  0.988157         0.231317  0.149487   
M             0.992040          0.985936  0.986839         0.227772  0.158787   

                 ...                  std                              \
                 ...     sink_last_ncites            sink_prev_ncites   
is_self_cite     ...                    0          1                0   
gender           ...                                                    
-                ...           169.724243  16.782538      6578.146046   
F                ...           160.320727  14.654411      5930.518831   
M                ...           153.001427  19.574211      5330.737559   

                                                                       \
                         auth_last_npapers           auth_prev_papers   
is_self_cite           1                 0         1                0   
gender                                                                  
-              98.974094          7.257212  8.087335        80.955529   
F              72.431688          4.723009  5.231927        55.122733   
M             104.447441          8.274032  9.395430       108.078080   

                                                  
                              jj_sim              
is_self_cite           1           0           1  
gender                                            
-             106.006992  107.416941  196.201352  
F              66.621107  112.665865  169.739648  
M             127.038444   91.100934  151.409221  

[3 rows x 172 columns]

In [7]:
df_t_last.T.unstack(level=0).to_csv("Last_Author_gender_self_cites_means.txt", sep="\t")

In [8]:
with pd.option_context('display.max_rows', 100,
                       'display.max_columns', 20,
                       'display.precision', 3
                      ):
    display(df_t_last.T.unstack(level=0))

gender                                    -                    F             \
                                       mean        std      mean        std   
                     is_self_cite                                             
source_year          0             2003.669      1.110  2003.634      1.112   
                     1             2003.669      1.112  2003.616      1.114   
source_n_mesh        0               12.921      5.690    12.995      5.410   
                     1               13.370      6.102    13.705      5.576   
source_is_eng        0                0.979      0.145     0.985      0.122   
                     1                0.996      0.066     0.994      0.080   
source_is_journal    0                0.987      0.115     0.987      0.115   
                     1                0.990      0.102     0.988      0.108   
source_is_review     0                0.171      0.377     0.231      0.422   
                     1                0.121      0.326     0.149      0.357   
source_is_case_rep   0                0.046      0.210     0.032      0.176   
                     1                0.013      0.111     0.011      0.105   
source_is_let_ed_com 0                0.013      0.115     0.014      0.115   
                     1                0.011      0.104     0.012      0.110   
source_T_novelty     0               21.133     12.993    21.882     12.990   
                     1               19.388     12.091    19.907     12.071   
source_V_novelty     0             3534.553  11270.689  3773.764   8360.934   
                     1             3097.273  13115.467  3109.543   6873.480   
source_PT_novelty    0                2.593      5.319     2.810      5.526   
                     1                2.096      4.570     2.236      4.731   
source_PV_novelty    0               10.314    198.982    14.104    392.588   
                     1                9.777    329.167     9.857    358.988   
source_ncites        0               46.306     59.037    56.770     95.506   
                     1               39.348     36.907    44.151     40.060   
source_n_authors     0                5.176      3.610     4.827      3.125   
                     1                5.300      5.136     5.035      3.475   
sink_year            0             1995.291      7.777  1995.531      7.548   
                     1             1999.435      4.195  1998.849      4.363   
sink_n_mesh          0               12.621      5.579    12.843      5.349   
                     1               13.382      5.947    13.759      5.327   
sink_n_mesh_ex       0               57.827     24.093    59.082     22.996   
                     1               61.242     26.191    63.276     23.312   
sink_is_eng          0                0.995      0.073     0.996      0.065   
                     1                0.997      0.054     0.996      0.059   
sink_is_journal      0                0.982      0.132     0.981      0.135   
                     1                0.990      0.101     0.990      0.097   
sink_is_review       0                0.148      0.355     0.151      0.358   
                     1                0.062      0.242     0.076      0.265   
sink_is_case_rep     0                0.038      0.191     0.031      0.174   
                     1                0.013      0.114     0.012      0.111   
sink_is_let_ed_com   0                0.018      0.133     0.019      0.136   
                     1                0.010      0.101     0.010      0.098   
sink_T_novelty       0               17.273     11.708    17.633     11.678   
                     1               17.166     11.370    17.394     11.234   
sink_V_novelty       0             2529.747   9435.920  2578.396   7800.286   
                     1             2404.246   5600.294  2391.182  12136.917   
sink_PT_novelty      0                1.927      4.580     1.961      4.590   
                     1                1.558      3